<img src="assets/bootcamp.png">

# **LTI&trade; Advantage** bootcamp notebook for Tool

Ping me if you have any question or feedback: claude.vervoort@gmail.com

This bootcamp code is available on: https://github.com/claudevervoort-perso/ltibootcamp, if you want to run it on your local laptop/network.

Availaibility of this notebook online is highly dependent on the depletion rate of my google free tier :)... but it is easy to get running locally.

## Introduction

The notebook shows how to interact with the LTI Advantage ecosystem from a tool implementer viewpoint. It interacts with an actual test server which has been built as a platform simulator to support this notebook. 

<img src="assets/bootcamp_arch.png" width="60%">

## Limitation

The test tool platform cannot launch into the bootcamp. As a workaround, the test tool has APIs to get the launch data that would have been included in an actual launch.


In [ ]:
# This notebook queries an actual tool platform test server. It needs its location:
platform_url='http://ltiplatform.zeedeeyou.com'

## Setup

### Import the python libraries needed by the tool

Here we just import the libraries that will be needed in this notebook, define some utility functions and constants.

In [ ]:
import requests
import json
import jwt
import base64
import re
from time import time, sleep
from datetime import datetime, timedelta
from cryptography.hazmat.primitives.asymmetric.rsa import RSAPublicNumbers
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.serialization import Encoding, PublicFormat
from IPython.display import display, Markdown, HTML, Javascript

def decode_int(b64value):
    return int.from_bytes(base64.urlsafe_b64decode(b64value), byteorder='big')

# for concise code, return full claim 
def fc(claim, s=None):
    suffix = ('-'+s) if s else ''
    return 'https://purl.imsglobal.org/spec/lti{0}/claim/{1}'.format(suffix, claim)

def scope(scope, s=None):
    suffix = ('-'+s) if s else ''
    return 'https://purl.imsglobal.org/spec/lti{0}/scope/{1}'.format(suffix, scope)
    
def md(mdt):
    display(Markdown(mdt))
    
md_buffer = ''

def mdb(mdt=None):
    global md_buffer
    if mdt:
        md_buffer = md_buffer + '\n' + mdt
    else:
        md(md_buffer)
        md_buffer = ''


## Deploying tool and establishing keys

### get the tool deployment info to use in this notebook

First we need to get a new tool deployment from the server for this notebook instance to use.
Each tool must have a `client_id` and a private key that will be used to interact with the platform services and send messages back to the platform. The `client_id` is used to for security purposes. A trust may be shared across multiple deployments of the same tool in a platform, so a `deployment_id` is also communicated to identify the actual deployment of that tool.

It also needs the keyset URL that exposes the platform public keys needed to validate the incoming messages.

While this information is required for each tool, how it is obtained by the tool is NOT currently part of the LTI specifications.

<img src="assets/security.png" width="70%">

In [ ]:
r = requests.get(platform_url + '/newtool')
tool_info = json.loads(r.text)

md('### Tool information')
md('Here is the tool information generated for this notebook. It is stored in ```tool_info``` variable.')
md('```json\n'+ json.dumps(tool_info, indent=4)+'```')

### Getting the public keys from the platform

In order to validate the various messages we will receive from the platform, we need to get the key sets. We'll also transform the keys to the `PEM` format that is used by the `jwt` module to decode the messages.

The test platform exposes its keys in a keyset format at a well-know location (.well-known/jwks.json). Other platform might just communicate the keyset url as part of the tool deployment information.

In [ ]:
keyset = json.loads(requests.get(tool_info['keyset_url']).text)

md('#### Platform keyset')
md('```json\n'+ json.dumps(keyset, indent=4)+'```')

platform_keys = {}

# let's transform as a map for ease of use, and just the PEM because this is what is used by JWT lib
for key in keyset['keys']:
    public_key = RSAPublicNumbers(decode_int(key['e']), decode_int(key['n'])).public_key(default_backend())
    pem = public_key.public_bytes(Encoding.PEM, PublicFormat.SubjectPublicKeyInfo)
    platform_keys[key['kid']] = pem
    


## Deep Linking - Creating a Link
This section will use the deep linking specification to create a Resource Link to the platform. That resource link will be gradable and used in the following sections of that notebook.

[Deep Linking](https://www.imsglobal.org/specs/lticiv1p0) is a **UI flow**, it is an important piece that is sometimes missed on 1st reading. The user is redirected from the platform to the tool to pick or create one or multiple piece of content (often, LTI links), and the the tool redirects the UI back to the tool with the actual selection (or an empty payload if nothing was picked or created).

So there are 2 messages:

1. `LtiDeepLinkingRequest` from the platform to the tool to start the picking/create session. This is a typical platform launch that contains the context and the user information, and what kind of content items may be created in this flow (for example, this flow might indicate it only wants LTI links).
1. `LtiDeepLinkingResponse` from the tool back to the plaform using the `content_item_return_url` provided in the request.

Once a tool is added to a course, usually the first launch from the platform will be a Deep Linking request.

<img src="assets/deeplinking.png" width="60%">

### Setup: Getting a Deep Linking Request

This notebook is not a tool actually launched by the platform, so the test platform as way to give us the token that it would include in an actual HTTP POST request, so we can build a mock POST request including the parameter `post_data`.

In [ ]:
r = requests.get("{}/tool/{}/deeplinkingmessage".format(platform_url, tool_info['client_id']))

post_data = {
    'id_token':r.text
}

md('`id_token='+ r.text+'`')

### Task 1: Verify the JWT is properly signed

The first thing before to display to the user the picker/authoring interface to create the link is to validate this request is properly signed. This is done by decoding the JWT using public key from the platform.


In [ ]:
# Let's get the kid so we can get the proper public key

# should check ISS first to lookup keyset

encoded_jwt = post_data['id_token']
jwt_headers = jwt.get_unverified_header(encoded_jwt)

content_item_message = jwt.decode(encoded_jwt, 
                                  platform_keys[jwt_headers['kid']], 
                                  jwt_headers['alg'],
                                  audience = tool_info['client_id'])

md('#### Message properly signed! Decoded ContentItemSelectionRequest message:')
md('```json\n'+ json.dumps(content_item_message, indent=4)+'```')


### Task 2: extract the information needed to render the selector/authoring UI

If this is the first launch for the user or the course, as a tool you may prompt the user for setup information, including account linking or course setup. Ultimately the user will see the authoring or picking interface that will allow her to create or select the content items to be added to the course.

Some key attributes of the `ContentItemSelectionRequest` will drive the experience:

In [ ]:
# fc(claim) prefix the claim with http://imsglobal.org/lti/
mdb('1. What is the current course id? {0}'.format(content_item_message[fc('context')]['id']))
mdb('1. What is the current user id? {0}'.format(content_item_message['sub']))
is_instructor = len(list(filter(lambda x: 'instructor' in x.lower(), content_item_message[fc('roles')])))>0
mdb('1. Is this user an instructor? {0}'.format(is_instructor))
deep_linking_claim = content_item_message[fc('deep_linking_settings', 'dl')]
mdb('1. What kind of content item can be created? {0}'.format(deep_linking_claim['accept_types']))
mdb('1. Can I return more than one items to be added? {0}'.format(deep_linking_claim['accept_multiple']))
mdb('1. Will the user be prompted before to actually save the items? {0}'.format(not deep_linking_claim['auto_create']))
deep_linking_return_url = content_item_message[fc('launch_presentation')]['return_url']
mdb('1. Where should I redirect the browser too when done? {0}'.format(deep_linking_return_url))
mdb('1. Is there any data I must pass back to platform when I return? {0}'.format('data' in deep_linking_claim))
mdb()

### Task 3: building the response token

After the end of the interaction, so user is sent back to the platform throught a browser redirection using an HTTP POST containing the JWT `ContentItemResponse` message. In this case, we will return 2 LTI links, one being graded (since the request supports multiple content items).

Here we're creating the actual response token.

In [ ]:
## First let's create our 2 content items
## Note that the URLs are phony as for now there is now way to launch in the notebook anyway...
simple_link =  {
      "type": "ltilink",
      "url": "http://lti.bootcamp/item111",
      "window": {
        "targetName": "bootcamp-win"
      },
      "iframe": {
        "height": 890
      },
      "title": "A simple content item",
      "text": "Some long text",
      "icon": {
        "url": "http://lti.example.com/image.jpg",
        "width": 100,
        "height": 100
      },
      "custom": {
        "lab": "sim4e"
      }
}

assignment_link =  {
      "type": "ltilink",
      "title": "An assignment",
      "text": "Chemical lab sim",
      "window": {
        "targetName": "examplePublisherContent"
      },
      "iframe": {
        "height": 890
      },
      "icon": {
        "url": "http://lti.example.com/image.jpg",
        "width": 100,
        "height": 100
      },
      "custom": {
        "lab": "sim3a",
        "level": "easy"
      },
      "lineItem": {
        "scoreMaximum": 34,
        "label": "Chemical lab sim",
        "resourceId": "sim3a",
        "tag": "final_grade"
      }
}

now = int(time())

deep_linking_response = {
    "iss": tool_info['client_id'],
    "aud": content_item_message['iss'],
    "exp": now + 60,
    "iat": now,
    fc('message_type'): "DeepLinkingResponse",
    fc('version'): "1.3.0",
    fc('content_items', 'dl'): [
        simple_link, assignment_link
    ]
    
}

### Task 4: build the signed JWT

In [ ]:
deep_linking_response_token = jwt.encode(deep_linking_response, tool_info['webkeyPem'], 'RS256').decode()

print(deep_linking_response_token)

### Task 5: redirect the user back to the platform with the content item selection

Now that we the response token, let's do the actual HTML POST redirection to the platform. Note that because the platform supports `autocreate` there will be no prompt. The 2 items will be added directly to the course.


In [ ]:
# Let's start by adding the JWS security claims
content_item_response = {
    'iss': tool_info['client_id'] ,
    'aud': content_item_message['iss']
}

autosubmit_js = """
                var ltiForm = $('<form>');                
                ltiForm.attr('action', '{url}');
                ltiForm.attr('method', 'POST');
                ltiForm.attr('target', 'deeplinking_frame');
                $('<input>').attr({{
                    type: 'hidden',
                    name: 'jws_token',
                    value: '{token}'
                }}).appendTo(ltiForm);
                $('#deeplinking_frame').before(ltiForm);
                ltiForm.submit();
                ltiForm.remove();
                """

autosubmit_js = autosubmit_js.format(url=deep_linking_return_url, token=deep_linking_response_token)

display(HTML('<iframe id="deeplinking_frame" name="deeplinking_frame" style="height: 300px; width:100%"></iframe>'))
display(Javascript(data=autosubmit_js, 
                   lib="https://code.jquery.com/jquery-3.3.1.min.js"))

## Student Resource Link launch

Now that we have created resource links, let's handle a student launch from one of them. We're going to use a resource link with a **coupled** line item, so that we can use it to send a score back to the platform.

### Setup

The first thing we need, as with deep linking, is to get from the test platform the launch token which an actual tool would get in an actual HTML Form Post.

In [ ]:
# in automated run, force a sleep for the response above to have been processed
sleep(3)

# select an id from the ones displayed in the course platform in the IFrame above
# if not specified the platform will pick a resource to use

resource_link_id = ''
context_id = content_item_message[fc('context')]['id']

r = requests.get("{}/tool/{}/context/{}/studentlaunch?rlid={}".format(platform_url, 
                                                           tool_info['client_id'], 
                                                           context_id, 
                                                           resource_link_id))

post_data = {
    'id_token':r.text
}

md('`id_token='+ r.text+'`')

### Task 1: Decode the launch

Now, same as with the Deep Linking request, we decode the token:

In [ ]:
encoded_jwt = post_data['id_token']
jwt_headers = jwt.get_unverified_header(encoded_jwt)

student_launch = jwt.decode(encoded_jwt, 
                            platform_keys[jwt_headers['kid']], 
                            jwt_headers['alg'],
                            audience = tool_info['client_id'])

md('```json\n'+ json.dumps(student_launch, indent=4)+'```')

### Task 2: extract information to show the correct activity

The launch gives information about the user, her role, the course, but also which actual resource we want to launch into.

In [ ]:
# fc(claim) prefix the claim with http://imsglobal.org/lti/
mdb('1. Is this a resource link launch? {0}'.format(student_launch[fc('message_type')] == 'LTIResourceLinkLaunch'))
mdb('1. What is the id of the resource link that is launched? {0}'.format(student_launch[fc('resource_link')]['id']))
mdb('1. What is the name of the resource that is launched? {0}'.format(student_launch[fc('resource_link')]['title']))
mdb('1. What is the current course id? {0}'.format(student_launch[fc('context')]['id']))
mdb('1. What is the current user id? {0}'.format(student_launch['sub']))
is_learner = len(list(filter(lambda x: 'learner' in x.lower(), student_launch[fc('roles')])))>0
mdb('1. Is this user a student? {0}'.format(is_learner))
return_url = student_launch[fc('launch_presentation')]['return_url']
mdb('1. Where should I redirect the browser too when done? {0}'.format(return_url))
mdb('1. Which lab should be displayed? {0}'.format(student_launch[fc('custom')]['lab']))
ags_claim = student_launch[fc('endpoint', 'ags')]
mdb('1. Is there a gradebook column for this resource? {0}'.format('lineitem' in ags_claim))
mdb()


## Assignment and Grade Services: Posting a grade to a coupled line item

Now that the student has launched into a grading activity, eventually she will complete it. Let's assume this is an autograded quiz. At the end of the interaction, we're going to send a score.

<img src="assets/ags.png" width="60%">

The line item (gradebook column) was created declaratively by the platform when the resource link was created (at the end of the deep linking flow). The line item and the resource link are **coupled**. Due to this explicit relationship, the platform passes us directly the associated line item URL, which we will use to send scores. 

### Task 1: Get an access token

To be able to send a grade, or call any service on that matter, we must first get an access token. This is done by using a JWT based client grant flow [RFC-7523](https://tools.ietf.org/html/rfc7523).

Here we will re-use the token for the rest of the notebook, so we don't specify scope. If you intend to use the token only for a given operation, it is a good practice to scope it accordingly.

The grant type is [client_credentials](https://tools.ietf.org/html/rfc6749#section-1.3.4) as the trust is established between the tool and the platform. The current user and context are not considered.


In [ ]:
## Let's define auth functions we can re-use for other calls

def get_token(scope):
    display(scope)
    token_endpoint = tool_info['accesstoken_endpoint']

    now = int(time())

    assertion = {
        "iss": tool_info['client_id'],
        "aud": token_endpoint,
        "exp": now + 60,
        "iat": now,
        "jti": "{0}-{1}".format(tool_info['client_id'], now)
    }

    assertion_jwt = jwt.encode(assertion, tool_info['webkeyPem'], 'RS256').decode()

    return json.loads(requests.post(token_endpoint, data = {
        'client_assertion': assertion_jwt,
        'grant_type': 'client_credentials',
        'scope': scope,
        'client_assertion_type': 'urn:ietf:params:oauth:client-assertion-type:jwt-bearer'
    }).text)

# We'll also need to create a proper header, so let's also create a function for that
def add_authorization(headers, access_token):
    b64token = base64.b64encode(access_token.encode('utf-8')).decode()
    headers.update({'Authorization': 'Bearer {0}'.format(b64token)})

    


In [ ]:
token_info = get_token('{0} {1}'.format(scope('result.readonly', s='ags'), scope('score', s='ags')))
md('#### Access token:')
md('```json\n'+ json.dumps(token_info, indent=4)+'```')

### Task 2: Publish a score

In [ ]:
# Scores in the subpath scores from lineitem.
def append_to_path(path, subpath):
    p = re.compile('lineitem($|\?|#)')
    return p.sub('lineitem/{0}\\1'.format(subpath), path)

scores_url = append_to_path(ags_claim['lineitem'], 'scores')

score = {
    'userId': student_launch['sub'],
    'scoreGiven': 9,
    'scoreMaximum': 10,
    'activityProgress': 'Completed',
    'gradingProgress': 'FullyGraded',
    'timestamp': datetime.utcnow().isoformat()
}

headers = {'Content-Type': 'application/vnd.ims.lis.v1.score+json'}
add_authorization(headers, token_info['access_token'])

r = requests.post(scores_url.encode(), headers=headers, data=json.dumps(score))

# let's check it was OK
r.raise_for_status()

md('The score was processed successfully be the back-end')




### Task 3: get the results

Let's not get the results to see our operation did actually succeed

In [ ]:
results_url = append_to_path(ags_claim['lineitem'], 'results')

headers = {'Content-Type': 'application/vnd.ims.lis.v2.resultcontainer+json'}
add_authorization(headers, token_info['access_token'])

r = requests.get(results_url.encode(), headers=headers)

# let's check it was OK
r.raise_for_status()

md('#### Current results for item')
md('```json\n'+ json.dumps(r.json(), indent=4)+'```')



### Task 4: marking a score as needing grading
Assignment and Grade services allow for more than just passing grade. Let's say we just want to mark a submission was done, and the instructor requires to manually grade the essay. In which case, we can post a Score indicating the status of the activity is completed, and the grading needs manual intervention using the `activityProgress` and `gradingProgress` attributes.

In [ ]:
score = {
    'userId': student_launch['sub'],
    'activityProgress': 'Completed',
    'gradingProgress': 'PendingManual',
    'timestamp': datetime.utcnow().isoformat()
}

headers = {'Content-Type': 'application/vnd.ims.lis.v1.score+json'}
add_authorization(headers, token_info['access_token'])

r = requests.post(scores_url.encode(), headers=headers, data=json.dumps(score))

# let's check it was OK
r.raise_for_status()

md('The score was processed successfully be the back-end')


We can see it now in the platform gradebook:

In [ ]:
platform_gradebook_url = '{0}/course/{1}/gradebook'.format(platform_url, student_launch[fc('context')]['id'])
display(HTML('<iframe src="{0}" style="height: 300px; width:100%"></iframe>'.format(platform_gradebook_url)))

## Assignment and Grade Services - Uncoupled

The assignment and grade services support an alternate way to create line items: rather than having the platform creates a line item when it creates a link, a tool may use the API to create standalone line items, i.e. uncoupled from any lti link (the API may optionaly include an ltiLinkId to tie the one or many line items to a single resource link, although platforms will often treat this association as weaker than a declarative flow).

Let's use the API to add a new line item, send some scores to it, then delete it.


### Task 1: get a properly scoped token

Same as before, we just need to now ask also for the scope to allow line item management.

In [ ]:
token_info = get_token(' '.join([scope('lineitem', s='ags'), scope('score', s='ags'), scope('result.readonly', 'ags')]))

md('#### Access token with all assignment and grade services scopes:')
md('```json\n'+ json.dumps(token_info, indent=4)+'```')

### Task 2: create a new line item and post score

The `lineitems` is the URL representating all the line items in the current context bound to our tool. We can do a GET to see all the existing items, and a POST to add a new one. Let's start by adding one:

In [ ]:
# note the specific mediatype
headers = {'Content-Type': 'application/vnd.ims.lis.v2.lineitem+json'}
add_authorization(headers, token_info['access_token'])

lineitem = {
    "resourceId": "bootcamp",
    "tag": "participation",
    "label": "Participation in Bootcamp",
    "scoreMaximum": 25
}

r = requests.post(ags_claim['lineitems'], headers=headers, data=json.dumps(lineitem))

r.raise_for_status()

uncoupled_item = r.json();

md('#### Successfully added line item:')
md('```json\n'+ json.dumps(uncoupled_item, indent=4)+'```')



In [ ]:
r = requests.get(ags_claim['lineitems'], headers=headers)

# let's check it was OK
r.raise_for_status()

md('#### Current line items for this tool in the context')
md('```json\n'+ json.dumps(r.json(), indent=4)+'```')

Sending the score to the uncoupled line item is the same as with the coupled one, the id being the line item URL.

In [ ]:
scores_url = append_to_path(uncoupled_item['id'], 'scores')

score = {
    'userId': student_launch['sub'],
    'scoreGiven': 10,
    'scoreMaximum': 10,
    'activityProgress': 'Completed',
    'gradingProgress': 'FullyGraded',
    'timestamp': datetime.utcnow().isoformat()
}

headers = {'Content-Type': 'application/vnd.ims.lis.v1.score+json'}
add_authorization(headers, token_info['access_token'])

r = requests.post(scores_url.encode(), headers=headers, data=json.dumps(score))

# let's check it was OK
r.raise_for_status()

md('The score was processed successfully be the back-end')


### Task 3: Updating the line item

Update is done by PUTting the updated definition to the line item  URL (id).

In [ ]:
# note the specific mediatype
headers = {'Content-Type': 'application/vnd.ims.lis.v2.lineitem+json'}
add_authorization(headers, token_info['access_token'])

lineitem = {
    "resourceId": "bootcamp",
    "tag": "participation",
    "label": "Participation (uncoupled)",
    "scoreMaximum": 30
}

r = requests.put(uncoupled_item['id'], headers=headers, data=json.dumps(lineitem))
                           
r.raise_for_status()

uncoupled_item = r.json();

md('#### Successfully updated line item:')
md('```json\n'+ json.dumps(uncoupled_item, indent=4)+'```')


You can also scroll back up where the platform UI is displayed and check in the gradebook to see the item is present (and scores have been rescaled).

### Task 4: Delete the uncoupled line item

Deleting is just issuing a DELETE command the line item URL.

In [ ]:
# no content type, it's a delete
headers = {}
add_authorization(headers, token_info['access_token'])

r = requests.delete(uncoupled_item['id'], headers=headers)
                           
r.raise_for_status()

md('#### Successfully deleted line item')


## Names and Role Provisioning Service

Or more simply, membership service, lets you get the roster of the course.

<img src="assets/membership.png" width="60%">

This service does not allow any modification of the course roster. At a minimum, the platform will apply the same privacy restrictions that applies to actual launches.

### Task 1: Get the full course roster

The Assignment and Grade Service endpoint is communicated in the LTI message under the claim `https://purl.imsglobal.org/lti/claim/arps`. The scope to request is `https://purl.imsglobal.org/lti/scope/nrps`.



In [ ]:
## Get An Access Token

membership_token = get_token(scope('contextmembership.readonly', s='nrps'))

## Request the enrollment

headers = {'Content-Type': 'application/vnd.ims.lis.v2.membershipcontainer+json'}
add_authorization(headers, membership_token['access_token'])
membership_url = student_launch[fc('namesroleservice', s='nrps')]['context_memberships_url']

r = requests.get(membership_url.encode(), headers=headers)

r.raise_for_status()

roster = r.json();

md('#### Successfully retrieved course roster:')
md('```json\n'+ json.dumps(roster, indent=4)+'```')


## Task 2: Get all changes since previous sync

Some platforms will include in their membership payload a `differences` url. This url, when present, allows the tool to update the roster based on the changes since last time the membership was returned, avoiding to transfer the full roster each time.

Use the differences url above to check for updates in roster.



In [ ]:
r = requests.get(roster['differences'].encode(), headers=headers)

r.raise_for_status()

roster_updates = r.json();

md('#### Successfully retrieved roster update:')
md('```json\n'+ json.dumps(roster_updates, indent=4)+'```')


## Task 3: get all students and paging

The service allows also to filter by roles using the `role` parameter. Both short and long version of roles can be used to filter (for example `Learner` and `http://purl.imsglobal.org/vocab/lis/v2/membership#Learner`). So let's limit the results to 3 students using the `limit` parameter.

In [ ]:
params = {'role':'Learner', 'limit': 3}

r = requests.get(membership_url.encode(), params=params, headers=headers)

r.raise_for_status()

roster = r.json();

md('#### Students only:')
md('```json\n'+ json.dumps(roster, indent=4)+'```')

# Substitution parameters - tailor your launch

Substitution parameters allow to have custom parameters resolve at runtime information from the platform. For example, if you would like to know the context history, i.e. from which course this course is a copy from, there is the substitution parameter: `$Context.id.history`. Substitution parameters always start will $ . If the platform supports it, the value will be substituted at runtime, otherwise it will remain `$Context.id.history`.

Substitution parameters really allow the tool to tailor its launches based on its particular needs. Substitution parameters may be set during the tool deployment. But it really shines with Deep Linking where the custom parameters can express the substitution parameters to resolve for the added LTI Link.

## Task 1: create an LTI Link with a line item and due date support

In this exercise, we're going to ask the platform to give us the due date of the activity (when a student is last authorized to submit) `$ResourceLink.submission.endDateTime` and the context history `$Context.id.history`. Since we're at it, we're also going to communicate an initial due date time using the `submission.endDateTime` property in the deep linking item.


In [ ]:
# Getting a deep linking request from the platform

r = requests.get("{}/tool/{}/deeplinkingmessage".format(platform_url, tool_info['client_id']))

post_data = {
    'id_token':r.text
}
encoded_jwt = post_data['id_token']
jwt_headers = jwt.get_unverified_header(encoded_jwt)

content_item_message = jwt.decode(encoded_jwt, 
                                  platform_keys[jwt_headers['kid']], 
                                  jwt_headers['alg'],
                                  audience = tool_info['client_id'])

# Adding an LTI link with a line item and end date

due_date = datetime.utcnow() + timedelta(days=7)

duedate_assignment_link =  {
      "type": "ltilink",
      "url": "http://lti.bootcamp/item111",
      "title": "Assignment with Dates",
      "text": "Do the assignment before the due date",
      "icon": {
        "url": "http://lti.example.com/image.jpg",
        "width": 100,
        "height": 100
      },
      "custom": {
        "lab": "sim3b",
        "dueDate": "$ResourceLink.submission.endDateTime",
        "contextHistory": "$Context.id.history"
      },
      "lineItem": {
        "scoreMaximum": 34,
        "label": "Chemical lab sim",
        "resourceId": "sim3a",
        "tag": "final_grade"
      },
      "submission": {
          "endDateTime": due_date.isoformat()
      },
      "window": {
        "targetName": "ltibootcamp-win"
      },
      "iframe": {
        "height": 890
      }
}

now = int(time())


deep_linking_response = {
    "iss": tool_info['client_id'],
    "aud": content_item_message['iss'],
    "exp": now + 60,
    "iat": now,
    fc("message_type"): "DeepLinkingResponse",
    fc("version"): "1.3.0",
    fc("content_items", s='dl'): [
        duedate_assignment_link
    ]
    
}

# and return to the platform with the selection

deep_linking_response_token = jwt.encode(deep_linking_response, tool_info['webkeyPem'], 'RS256').decode()

content_item_response = {
    'iss': tool_info['client_id'] ,
    'aud': content_item_message['iss']
}

autosubmit_js = """
                var ltiForm = $('<form>');                
                ltiForm.attr('action', '{url}');
                ltiForm.attr('method', 'POST');
                ltiForm.attr('target', 'deeplinking_frame_2');
                $('<input>').attr({{
                    type: 'hidden',
                    name: 'jws_token',
                    value: '{token}'
                }}).appendTo(ltiForm);
                $('#deeplinking_frame_2').before(ltiForm);
                ltiForm.submit();
                ltiForm.remove();
                """

autosubmit_js = autosubmit_js.format(url=deep_linking_return_url, token=deep_linking_response_token)

display(HTML('<iframe id="deeplinking_frame_2" name="deeplinking_frame_2" style="height: 300px; width:100%"></iframe>'))
display(Javascript(data=autosubmit_js, 
                   lib="https://code.jquery.com/jquery-3.3.1.min.js"))

## Task 2: launch the activity and verify substitution parameters

Now that we have created a link, let's get a student launch and see if on launch we can get the current due date time. Those could indeed have been changed on the platform, or that student may benefit from an actual extension.

In [ ]:
sleep(3)

In [ ]:
# select an id from the ones displayed in the course platform in the IFrame above
# if not specified the platform will pick a resource to use

resource_link_id = 'fec7ac2d-562f-11e8-a10d-c48e8ffb7857'
context_id = content_item_message[fc('context')]['id']

r = requests.get("{}/tool/{}/context/{}/studentlaunch?rlid={}&accomodation={}".format(platform_url, 
                                                           tool_info['client_id'], 
                                                           context_id, 
                                                           resource_link_id, 3))

post_data = {
    'id_token':r.text
}

encoded_jwt = post_data['id_token']
jwt_headers = jwt.get_unverified_header(encoded_jwt)

launch = jwt.decode(encoded_jwt, 
                            platform_keys[jwt_headers['kid']], 
                            jwt_headers['alg'],
                            audience = tool_info['client_id'])

md('Student `{0}` assignment is due on `{1}`'.format(launch['name'], launch[fc('custom')]['dueDate']))
md('The history of this course context is `{0}`'.format(launch[fc('custom')]['contextHistory']))
